# Constructing a Dobble card set

Every card of a Dobble game contains a fixed number $n$ of symbols. The set of cards is defined by the rules

1. Any two cards have exactly one symbol in common.
2. Any two symbols appear together on exactly one card.

It turns out that these requirements have an exact geometrical analogue. In two-dimensional affine geometry, the following two rules hold:

1. Any two non-parallel lines meet in exactly one point.
2. Any two points lie on (define) exactly one line. 

Since these characteristics depend only on the underlying vector-space, they particularly hold for vector-spaces over *finite fields* $\mathbb{F}_q$ with $q$ elements. So the strategy to construct a Dobble card game is as follows. Associate "card" with "line" and "symbol" with "point" and consider the affine plane $\mathbb{A}^2_q$ over the vector space $\mathbb{F}_q^2$. Next, construct every line in $\mathbb{A}^2_q$. Consider therefore that every line is defined by an equation
$$
    \mathbf{w}_i\cdot\mathbf{x} + b_j = 0 \qquad\qquad (1)
$$
for some $\mathbf{w}_i \in \mathbb{F}_q^2$ and $b_j\in \mathbb{F}_q$. In other words, each line is defined by the tuple $(\mathbf{w}_i, b_j)$ where $\mathbf{w}_i$ ranges over all different "directions" of the vector space. We can pick the set of $q+1$ representatives 
$$
    W := \left\{ (0,1)^T, (1,0)^T, (1,2)^T, \ldots, (1,q-1)^T \right\}
$$
for the directions $\mathbf{w}_i$. One can easily show with basic linear algebra that the $p^2 + p$ tuples $(\mathbf{w}_i, b_j) \in W \times \mathbb{F}_q$ indeed define the all the lines in $\mathbb{A}_q^2$. For every vector $\mathbf{x} = (x_1,x_2)^T$ in (1), if you fix any component $x_1$ or $x_2$, there is exactly one solution for the other one. It follows that every line contains exactly $q$ points. 

Now, two different lines intersect in exactly one point if they are _not parallel_, i.e. they have different direction vectors $\mathbf{w}$, whereas 
two lines wth the same directions don't have a point in common. To get hold of the parallel case, we assign a "point at infinity" $p_{\mathbf{w}_i} \not\in A(\mathbb{F_q})$ to every line with direction $\mathbf{w}_i$. With this convention, any two lines have exactly one point in common, whether they share the same direction or not. Finally, we add another line containing exactly all the "points at infinity". This makes sure that any two points lie on exactly one line. 

This finishes the construction of the Dobble card game. Substituting "card" with "line" and "point" with symbol, we see that required properties hold. Numberwise, the card set has the following properties:

* Every card contains $q+1$ symbols. (Every line contains $q$ normal points plus one "point at infinity".)
* There are $q^2 + q + 1$ different symbols. (The $q^2$ points in $\mathbb{A}_q^2$ plus $q+1$ "points at infinity".)
* There are $q^2 + q + 1$ different cards. (The $q^2 + q$ lines in $\mathbb{A}_q^2$ plus one "line at infinity".)

Note that the construction depends on the existence of a field $\mathbb{F}_q$ with $q$ elements. Such a field exists if and only if $q$ is a prime power, i.e. $q=p^n$ for some prime number $n$. For $n=1$ this is simply the residual ring $\mathbb{Z}/p\mathbb{Z}$ where the arithmetic is defined by modulo $p$. But in general, for $n>1$, the field $\mathbb{F}_q$ is not isomorphic to the the residual ring. 

In [ ]:
import random
import galois

class Line:
    def __init__(self, w, b, dim):
        self.w = w
        self.b = b
        self.dim = dim
        # The points on the line. 
        # All parallel lines (lines with equal w) have a "point at infinity" in common which, 
        # by convention, we denote with (-w_0, -w_1)
        self.points = [(-w[0],-w[1])]
        self.GF = galois.GF(dim)

    def contains(self, x):
        return (self.GF(self.w[0])*self.GF(x[0]) + self.GF(self.w[1])*self.GF(x[1]) + self.GF(self.b)) == 0      

    def print(self):
        print(f"w={self.w}x+{self.b}: Points: {self.points}")

class Dobble:

    # n is the order of the finite field. Therefore, n has to be a prime power, n=p^n for some prime p.
    def __init__(self, n):
        self.n = n
        # In a finite affine vector space over a field of order n there are:
        #   -- n+1 different directions: w=(0,1),(1,0),...(1,n-1)
        #   -- n different offsets: b=0,...,n-1
        # This makes n^2+n different lines defined by wx+b=0
        self.lines = [Line((0,1),b, n) for b in range(n)] + [Line((1,m),b,n) for m in range(n) for b in range(n)]
        self.points = [(x,y) for x in range(n) for y in range(n)]

        # Assign the points to the line
        # TODO: optimize! This is highly redundant. Simply compute $w*x$ for every direction and assign to the appropriate line.  
        for line in self.lines:
            for x in self.points:
                if line.contains(x):
                    line.points.append(x)
        
        # Add the "line at infinity" containing all "points at infinity"
        line_at_infinity = Line((0,0),0, n)
        line_at_infinity.points = [(0,-1)] + [(-1,-m) for m in range(n)]
        self.lines.append(line_at_infinity)

    def play(self):
        i,j = random.sample(range(len(self.lines)), 2)
        common_points = []
        for x in self.lines[i].points:
            for y in self.lines[j].points:
                if x==y:
                    common_points.append(x)
        
        print("New Game")
        self.lines[i].print()
        self.lines[j].print()
        print(F"Common point {common_points}")
        print("")

        
dobble = Dobble(7)

for i in range(10):
    dobble.play()
